# 作業（一）

最後編輯時間：2021/10/10

定義一個 document class 物件，初始值是給定資料夾路徑。

使用方法 read 來讀取檔案，針對 xml 以及 json 檔案分別處理，輸出一個 dataframe ，
內容包含檔案名稱（file），擷取的字串內容（content），字串總數（character），單字總數（word），
單字數量（token），多少句子（sentence），幾個篇幅（paragraph）。

使用方法 search 來根據指定的關鍵字，對指定路徑下的文件進行特定字詞搜尋。

In [21]:
import os, json
from xml.etree import ElementTree as et
import pandas, re
from nltk.stem import PorterStemmer

In [22]:
class document:

    ##  指定文件的資料夾。
    def __init__(self, folder):
        
        self.folder = folder
        return
    
    ##  根據資料夾去讀取文件資料。
    def read(self):

        ##  輸出一個 dataframe 。 
        group = {
            'file' : [], 
            'content': [],
            'character':[],
            'word':[],
            'word stem (unique)':[],
            'sentence':[],
            'paragraph':[], 
        }
        pass

        ##  塑膠袋。
        cache = {}
        for number, name in enumerate(os.listdir(self.folder), 1):

            path = os.path.join(self.folder, name)
            if('xml' in name):

                data = et.parse(path)
                root = data.getroot()

                ##  每個文件中有幾篇文章。
                paragraph = len(root.findall('PubmedArticle'))

                ##  取出所有字元，將多餘的空白符號、換行符號刪掉，
                ##  第一個空格刪掉。
                string = " ".join([e.text for e in root.iter()])
                string = re.sub('\s+', " ", string)
                string = string[1:]

                ##  計算字元數量。
                character = len(string)

                ##  計算單字數量。
                ##  去掉逗號、分號、句號、驚嘆號、問號、括號、冒號。
                cache['word'] = re.sub('[,.;:?()!]', "", string).split(" ")
                word = len(cache['word'])

                ##  重複的單字不計，總共有幾個單字。
                porter = PorterStemmer()
                cache['word stem'] = [porter.stem(i) for i in cache['word']]
                token = len(set(cache['word stem']))

                ##  計算句子數量。
                ##  非逗號、句號、問號、驚嘆號、分號刪除，
                ##  將多餘空白刪除，包含第一個字元的空白。
                ##  根據空白進行切割。
                cache['string'] = re.sub("[^,.!?;]", " ", string)
                cache['string'] = re.sub('\s+', " ", cache['string'])
                cache['string'] = cache['string'][1:]
                sentence = len(cache['string'].split(" "))
                pass

            if('json' in name):

                with open(path, 'r') as paper:

                    data = json.load(paper)
                    pass

                string = []
                for i in data:

                    for _, v in i.items():

                        string += [v]        
                        pass

                    pass

                ##  該文件中有幾篇推文。
                paragraph = len(data)

                ##  取出所有字元，將多餘的空白符號、換行符號刪掉，
                ##  第一個空格刪掉。
                string = " ".join(string)
                string = re.sub('\s+', " ", string)
                string = string[1:]

                ##  計算字元數量。
                character = len(string)

                ##  計算單字數量。
                ##  去掉逗號、分號、句號、驚嘆號、問號、括號、冒號。
                cache['word'] = re.sub('[,.;:?()!]', "", string).split(" ")
                word = len(cache['word'])

                ##  重複的單字不計，總共有幾個單字。
                porter = PorterStemmer()
                cache['word stem'] = [porter.stem(i) for i in cache['word']]
                token = len(set(cache['word stem']))

                ##  計算句子數量。
                ##  非逗號、句號、問號、驚嘆號、分號刪除，
                ##  將多餘空白刪除，包含第一個字元的空白。
                ##  根據空白進行切割。
                cache['string'] = re.sub("[^,.!?;]", " ", string)
                cache['string'] = re.sub('\s+', " ", cache['string'])
                cache['string'] = cache['string'][1:]
                sentence = len(cache['string'].split(" "))
                pass

            group['file'] += [name]
            group['content'] += [string]
            group['character'] += [character]
            group['paragraph'] += [paragraph]
            group['word'] += [word]
            group['word stem (unique)'] += [token]
            group['sentence'] += [sentence]
            pass
        
        self.data = pandas.DataFrame(group)
        print("load {} files...".format(number))
        return
    
    def search(self, keyword):

        output = {
            "file":[],
            "exist":[],
            "number":[]
        }
        pass
    
        for _, item in self.data.iterrows():

            output['file'] += [item['file']]
            output['exist'] += [bool(re.search(keyword, item['content']))]
            output['number'] += [len(re.findall(keyword, item['content']))]
            pass
        
        output = pandas.DataFrame(output)
        return(output)

    pass

In [23]:
doc = document(folder = 'demo_data')
doc.read()

load 6 files...


In [24]:
doc.data

,file,content,character,word,word stem (unique),sentence,paragraph
0,test6.json,SWHealth NSW Health https://twitter.com/NSWHea...,456,63,50,7,1
1,test5.json,regAbbott_TX Greg Abbott https://twitter.com/G...,449,55,47,11,1
2,test4.json,andPaul Senator Rand Paul https://twitter.com/...,360,47,42,4,1
3,test1.xml,33053381 2020 12 21 2021 01 10 1879-0712 889 2...,3600,473,267,100,1
4,test2.xml,33342635 2021 01 27 2021 05 21 1873-2518 39 4 ...,8794,1176,560,278,1
5,test3.xml,32512530 2020 12 10 2021 01 10 1876-2026 53 20...,4779,618,337,155,1


In [28]:
doc.search(keyword='Covid')

,file,exist,number
0,test6.json,False,0
1,test5.json,False,0
2,test4.json,True,2
3,test1.xml,False,0
4,test2.xml,False,0
5,test3.xml,False,0


In [7]:
doc.search(keyword='covid-19')

,file,exist,number
0,test6.json,False,0
1,test5.json,False,0
2,test4.json,False,0
3,test1.xml,False,0
4,test2.xml,False,0
5,test3.xml,False,0


In [8]:
doc.search(keyword='➡️')

,file,exist,number
0,test6.json,False,0
1,test5.json,False,0
2,test4.json,False,0
3,test1.xml,False,0
4,test2.xml,False,0
5,test3.xml,False,0


In [9]:
doc.search(keyword='Wei Luoqun L')

,file,exist,number
0,test6.json,False,0
1,test5.json,False,0
2,test4.json,False,0
3,test1.xml,False,0
4,test2.xml,False,0
5,test3.xml,False,0


In [10]:
doc.search(keyword='Wang Hongyun H')

,file,exist,number
0,test6.json,False,0
1,test5.json,False,0
2,test4.json,False,0
3,test1.xml,False,0
4,test2.xml,False,0
5,test3.xml,False,0


In [11]:
doc.search(keyword='Wang')

,file,exist,number
0,test6.json,False,0
1,test5.json,False,0
2,test4.json,False,0
3,test1.xml,False,0
4,test2.xml,False,0
5,test3.xml,False,0
